# Homework 1: Decision tree and K-nearest neighbor



This assignment is due on Canvas by **11:59pm on Friday, March 31**. 
Your solutions to theoretical questions should be done in Markdown/MathJax directly below the associated question.
Your solutions to computational questions should include any specified Python code and results 
as well as written commentary on your conclusions.
Remember that you are encouraged to discuss the problems with your instructors and classmates, 
but **you must write all code and solutions on your own**. For a refresher on the course **Collaboration Policy** click [here](https://canvas.uchicago.edu/courses/42240).

**NOTES**: 

- Do **NOT** load or use any Python packages that are not available in Anaconda with Python 3.9. 
- Some problems with code may be autograded.  If we provide a function API, **do not** change it.  If we do not provide a function API, then you're free to structure your code however you like. 
- Submit only this Jupyter notebook to Canvas.  Do not compress it using tar, rar, zip, etc. 

**Acknowledgment**: Noah Smith, Chris Ketelsen

**Name**: Yujie Jiang

In [2]:
import math
import pickle
import gzip
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
%matplotlib inline

### [60 points] Problem 1 - Decision tree
***

Consider the problem of predicting whether a person has a college degree based on age, salary, and Chicago residency. 
The dataset looks like the following.

| Age   | Salary         | Chicago Residency      | College degree| 
|:------:|:------------:| :-----------:|---:|
| 27 | 41,000 | Yes | Yes |
| 61 | 52,000 | No | No |
| 23 | 24,000 | Yes | No |
| 29 | 77,000 | Yes | Yes |
| 32 | 48,000 | No | Yes |
| 57 | 120,000 | Yes | Yes |
| 22 | 38,000 | Yes | Yes |
| 41 | 45,000 | Yes | No |
| 53 | 26,000 | No | No |
| 48 | 65,000 | Yes | Yes |


**Part A [5 points]**: Convert the above table to data. Two variables should be created:
        
1. $x$ is a $10*3$ matrix that contains the data from columns 0, 1, and 2. Chicago residency is represented by 1 (yes) and 0 (no).
2. $y$ contains the labels (college degree), 1 (yes) and 0 (no).

In [14]:
x = np.array([[27, 41000, 1], [61, 52000, 0], [23, 24000, 1], [29, 77000, 1], [32, 48000, 0],
     [57, 120000, 1], [22, 38000, 1], [41, 45000, 1], [53, 26000, 0], [48, 65000, 1]])
y = np.array([1, 0, 0, 1, 1, 1, 1, 0, 0, 1])

**Part B [10 points]:** Criteria for choosing a feature to split.

**[4 points]** We start with no splitting. Assuming that our algorithm is deterministic, what is the smallest number of mistakes we can make if we do not use any of the features and what is the algorithm? (**Write your answer in the Markdown cell below.**)

The algorithm behind which we don't use any feature is just random guess, thus it is more likely for us to make mistakes and the smallest number of mistakes we can make will be generally high. However, exact value of the smallest number of mistakes should be depends. 

**[6 points]** We start by considering the variable *Chicago residency*. The first criteria is based on the number of mistakes. We need to build a contingency table between Chicago residency and college degree.

How many mistakes will we make if we split based on Chicago residency? (**Answer below by finishing the code.**)

In [15]:
def get_error_in_leaf(y, ids):
    """
    Returns the errors in a leaf node of a decision tree.
    This function can be used to answer the previous question automatically.
    
    :@param y: all labels
    :@param ids: the subset of indexes in the leaf node
    """
    zero_count = 0
    one_count = 0
    leaf = y[ids]
    for label in leaf:
        if label == 0:
            zero_count += 1
        elif label == 1:
            one_count += 1
    if zero_count > one_count:
        return one_count
    else:
        return zero_count 

def error_criteria(y, root, left_child, right_child):
    """
    Returns the number of errors if we split the root into the left child and the right child.
    
    :@param y: all labels
    :@param root: indexes of all the data points in the root
    :@param left_child: the subset of indexes in the left child
    :@param right_child: the subset of indexes in the right child
    """
    left_error = get_error_in_leaf(y, left_child)
    right_error = get_error_in_leaf(y, right_child)
    return left_error + right_error

def value_split_binary_feature(x, y, fid, root, criteria_func):
    
    root = root.tolist()
    left_child = [i for i in root if x[i, fid] == 0]
    right_child = [i for i in root if x[i, fid] == 1]
    return criteria_func(y, root, left_child, right_child)

# Chicago residency should correpsond to the third column in your data x
fid = 2
root = np.array(list(range(len(y)))) # root includes all data points
mistakes = value_split_binary_feature(x, y, fid, root, error_criteria)

In [16]:
# This cell is for grading purposes only; please ignore

**Part C [20 points]** Alternatively, we can use entropy and information gain to split the data. In this part, you will manually determine the first split in a decision tree. Please do not use code in your calculations for part C.

**[4 points]** To get familiar with MathJax, please write the equations necessary to compute entropy and information gain if we split data $D$ into $D_1$ and $D_2$. **Write your answer in the Markdown cell below.**

Entropy: $$H(x) = -p_i *\log_2 p_i - (1-p_i)*\log_2 (1-p_i)$$
Information Gain: $$IG(D,D_i) = H(D) - \frac{X_l}{X_D}*H(X_l) - \frac{X_r}{X_D}*H(X_r)$$

**[4 points]** What is the entropy for College Degree? Please show your work. Were you expecting a result around this number? Why?

Entropy for College Degree: $$ -\frac{3}{5} * log_2 \frac{3}{5} -\frac{2}{5} * log_2 \frac{2}{5} \approx 0.98 $$
I exapected it to be a number around it because the College Degree is almost equally balanced, not much infomation can be given. 

**[4 points]** What is the information gain for College Degree if you split the observations on the Chicago Residency attribute? Please show your work.

$$ H(X_l) = -\frac{1}{3} * log_2 \frac{1}{3} -\frac{2}{3} * log_2 \frac{2}{3} \approx 0.918 $$

$$ H(X_r) = - \frac{2}{7} * log_2 \frac{2}{7} -\frac{5}{7} * log_2 \frac{5}{7} \approx 0.863 $$

$$ IG = 0.98 - \frac{3}{10} * 0.918 - \frac{7}{10} * 0.863 \approx 0.1005$$


**[4 points]** One way to deal with continuous (or ordinal) features like Age and Salary is to define binary features based on thresholding. For example, you might convert ages to 0 if Age is less than or equal to 50 and 1 otherwise. What is the information gain for College Degree if we split the observations based on the Salary attribute with a threshold of \$50,000? Please show your work.

(Later in this problem, we will write code to determine whether a number other than \$50,000 might be the optimal threshold for Salary. For now, just assume that a threshold of \\$50,000 is optimal.)

$$ H(X_l) = -\frac{1}{2} * log_2 \frac{1}{2} -\frac{1}{2} * log_2 \frac{1}{2} \approx 0 $$

$$ H(X_r) = - \frac{1}{4} * log_2 \frac{1}{4} -\frac{3}{4} * log_2 \frac{3}{4} \approx 0.811$$

$$ IG = 0.98 - \frac{3}{5} * 0 - \frac{2}{5} * 0.811 \approx 0.656$$

**[4 points]** Based on the information gain calculations, should the first split in our decision tree be on Chicago Residency or on Salary with a \$50,000 threshold? Is this the answer you expected based on the data counts? Why or why not?

We should firstly split our decision tree by Salary with a $50,000 threshold, because it generates higher information gain. 
It is not as expected, it is hard to tell which one is better if only based on the data counts. 

**Part D [10 points]** Now we will write a function for computing information gain. Use log2 for entropy computation.

In [17]:
def entropy(y, ids):
    """
    Returns the entropy in the labels for the data points in ids.
    
    :@param y: all labels
    :@param ids: the indexes of data points
    """
    zero_count = 0
    one_count= 0
    leaf = y[ids]

    if len(ids) == 0: # deal with corner case when there is no data point.
        return 0
    for label in leaf:
        if label == 0:
            zero_count += 1 
        else:
            one_count += 1
    
    zero_prob = zero_count/len(ids)
    one_prob = one_count/len(ids)
    entropy = -zero_prob * math.log2(zero_prob) - one_prob * math.log2(one_prob)
    return entropy
    
def information_gain_criteria(y, root, left_child, right_child):
    """
    Returns the information gain by splitting root into left child and right child.
    
    :@param y: all labels
    :@param root: indexes of all the data points in the root
    :@param left_child: the subset of indexes in the left child
    :@param right_child: the subset of indexes in the right child
    """
    left_div = len(left_child)/len(root)
    right_div = len(right_child)/len(root)
    info_gain = entropy(y, root) - left_div * entropy(y, left_child) - right_div * entropy(y, right_child)
    return info_gain
    
fid = 2
root = np.array(list(range(len(y)))) # root includes all data points
info_gain = value_split_binary_feature(x, y, fid, root, information_gain_criteria)    

In [18]:
# This cell is for grading purposes only; please ignore

**Part E [15 points]**: Deal with continuous features.
    
**[10 points]** Complete the following code:

In [19]:
def value_split_continuous_feature(x, y, fid, root, criteria_func=information_gain_criteria):
    """
    Return the best value and its corresponding threshold by splitting based on a continuous feature.

    :@param x: all feature values
    :@param y: all labels
    :@param fid: feature id to split the tree based on
    :@param root: indexes of all the data points in the root
    :@param criteria_func: the splitting criteria function
    """
    best_value, best_thres = 0, 0

    return best_value, best_thres

root = np.array(range(len(y))) # root includes all data points
fid = 0
age_value, age_thres = value_split_continuous_feature(x, y, fid, root, information_gain_criteria)
fid = 1
salary_value, salary_thres = value_split_continuous_feature(x, y, fid, root, information_gain_criteria)

In [20]:
# This cell is for grading purposes only; please ignore

In [21]:
# This cell is for grading purposes only; please ignore

**[5 points]** Based on the current information gain by splitting different features, if we build a decision stump (decision tree with depth 1) greedily, which feature should we choose? Why? **Write down your answer in the Markdown cell below.**

YOUR ANSWER HERE

**Extra credit [5 points]**: You now have all the ingredients to build a decision tree recursively. You can build a decision tree of depth two and report its classification error on the training data and the tree.

In [22]:
class LeafNode:
    """
    Class for leaf nodes in the decision tree
    """
    
    def __init__(self, label, count, total):
        """
        :@param label: label of the leaf node
        :@param count: number of data points with class 'label' falling in this leaf
        :@param count: number of datapoints of any label falling in this leaf
        """
        self.label = label
        self.count = count
        self.total = total
        
    def predict(self, x):
        """
        Return predictions for features x

        :@param x: feature values
        """
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def display(self, feat_names, out_str, depth=0):
        """
        Display contents of a leaf node
        """
        prefix = '\t'*depth
        error = 1.0 - self.count / float(self.total)
        out_str += f'{prefix}leaf: label={self.label}, error={error} ({self.count}/{self.total} correct)\n'
        return out_str
    
class TreeNode:
    """
    Class for internal (non-leaf) nodes in the decision tree
    """
    def __init__(self, feat_id, feat_val):
        """
        :@param feat_id: index of the feature that this node splits on
        :@param feat_val: threshold for the feature that this node splits on
        """
        self.feat_id = feat_id
        self.feat_val = feat_val
        self.left = None
        self.right = None
    
    def split(self, x, root):
        """
        Given the datapoints falling into current node, return two arrays of indices in x corresponding to the
        left and right subtree
        
        :@param x: all feature values
        :@param root: indexes of all the data points in the current node
        """
        root = np.array(root)
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def predict(self, x):
        """
        Return an array of predictions for given 'x' for the current node
        
        :@param x: datapoints
        """
        assert self.left is not None and self.right is not None, 'predict called before fit'
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def display(self, feat_names, out_str, depth=0):
        """
        Display contents of a non-leaf node
        """
        prefix = '\t'*depth
        out_str += f'{prefix}{feat_names[self.feat_id]}\n'
        out_str += f'{prefix}x <= {self.feat_val}\n'
        out_str = self.left.display(feat_names, out_str, depth=depth+1)
        out_str += f'{prefix}x > {self.feat_val}\n'
        out_str = self.right.display(feat_names, out_str, depth=depth+1)
        return out_str

class DecisionTree:
    """
    Class for the decision tree
    """
    def __init__(self, max_depth=1, criteria_func=information_gain_criteria, binary_feat_ids=[]):
        """
        :@param max_depth: Maximum depth that a decision tree can take
        :@param criteria_func: criteria function to split features
        :@param binary_feat_id: list of indexes of binary features
        """
        self.max_depth = max_depth
        self.criteria_func = criteria_func
        self.binary_feat_ids = binary_feat_ids
        self.root = None
        self.x = None
        self.y = None
        
    def fit(self, x, y):
        """
        Fit a tree to the given dataset using a helper function
        """
        self.x = x
        self.y = y
        self.root = self.fit_helper(np.array(list(range(self.x.shape[0]))))
    
    def fit_helper(self, root, depth=1):
        """
        Recursive helper function for fitting a decision tree
        Returns a node (can be either LeafNode or TreeNode)
        
        :@param root: array of indices of datapoints which fall into the current node
        :@param depth: current depth of the tree being built 
        """
        
        """
        Strategy:
        1. If current partition is pure i.e. labels corresponding to all indices in root are the same
           OR the maximum depth has been reached, stop building the tree and return a LeafNode
        2. If not, find out the best feature to split on along with the threshold, create a TreeNode and 
           recursively call fit_helper on the two splits (You can assume the threshold for a binary feature 
           to be 0.5). Finally, return the current node 
        """
        
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def predict(self, x):
        """
        Return predictions for a given dataset  
        """
        assert self.root is not None, 'fit not yet called'
        # YOUR CODE HERE
        raise NotImplementedError()
    
    
    def display(self, feat_names):
        assert self.root is not None, 'fit not yet called'
        out_str = ""
        out_str = self.root.display(feat_names, out_str)
        return out_str

In [23]:
# This cell is for grading purposes only; please ignore
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


### [40 points] Problem 2 - KNN for Crime Prediction 
***

In this problem, you will implement a K-Nearest Neighbors framework to predict the type of crime reported based on an incident's location.

Download [reported crime data for 2021](https://data.cityofchicago.org/Public-Safety/Crimes-2021/dwme-t96c) from the Chicago Open Data Portal (we already prepared it in the data folder, but would you to know where the source was).

**Part A [5 points]**: Load the data.  We will only use three columns of the dataset: Primary Type, Latitude, and Longitude.
- Be sure to drop any observations that are missing latitude and/or longitude.
- To reduce run time, only keep incidents reported as one of the the four most common crime types ('THEFT', 'BATTERY', 'CRIMINAL DAMAGE', 'ASSAULT'). 
- Use the train_test_split function from Scikit-Learn to split the data into training and validation sets.  Set random_state=123 and test_size=0.2.
- Finally, explore the training and validation sets and answer the following questions: 
  - How many total observations are in the training set? 
  - How many total observations are in the validation set? 

In [ ]:
# Write code for answering the questions in Part A and then put your answer in the Markdown cell below.
# Make sure to set each of the variables below to the correct value. Do not rename the variables.
N_training_examples = None
N_validation_examples = None

crime_data = pd.read_csv("")

NotImplementedError: 

In [ ]:
# This cell is for grading purposes only; please ignore

YOUR ANSWER HERE

**Optional:**  If you'd like to run the following code chunk to render a map of the incidents, you will need to download the [neighborhood boundaries data](https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Neighborhoods/bbvz-uum9) from the Chicago Data Portal (again, we have provided a version in the data folder).

In [ ]:
# Import library
import geopandas as gpd

# Load boundaries
file = open('data/boundaries.geojson')
boundaries = gpd.read_file(file) 

# Convert full crime dataset from pandas to geopandas
crime_gdf = gpd.GeoDataFrame(crime_df, geometry=gpd.points_from_xy(crime_df['Longitude'], crime_df['Latitude']),
                             crs='EPSG:4326')

# Make plot
fig, ax = plt.subplots(1, figsize=(12, 16))
boundaries.plot(ax=ax, color='white', edgecolor='black')
colors = {'THEFT' : 'red', 'BATTERY' : 'blue', 'CRIMINAL DAMAGE' : 'yellow', 'ASSAULT' : 'gray'}
grouped = crime_gdf.groupby('Primary Type', sort=True)
for key in colors:
    group = grouped.get_group(key)
    group.plot(ax=ax, label=key, color=colors[key], markersize=0.3)
ax.legend(markerscale=10)
ax.axis('off')
ax.set_title('Reported Incidents in Chicago, 2021', fontdict={'fontsize': '25', 'fontweight' : '3'});

In [ ]:
class KNN:
    """
    Class to store data for regression problems 
    """
    def __init__(self, X_train, y_train, K=5, distance_weighted=False):
        """
        Creates a kNN instance

        :param X_train: Training data input in 2D ndarray 
        :param y_train: Training data output in 1D ndarray 
        :param K: The number of nearest points to consider in classification
        :param distance_weighted: Bool indicating whether to use distance weighting
        """
        
        # Import and build the BallTree on training features 
        from sklearn.neighbors import BallTree
        self.balltree = BallTree(X_train)
        
        # Cache training labels and parameter K 
        self.y_train = y_train
        self.K = K 
        
        # Boolean flag indicating whether to do distance weighting 
        self.distance_weighted = distance_weighted
        
    def majority(self, neighbor_indices, neighbor_distances=None):
        """
        Given indices of nearest neighbors in training set, return the majority label. 
        Break ties by considering 1 fewer neighbor until a clear winner is found. 

        :param neighbor_indices: The indices of the K nearest neighbors in self.X_train 
        :param neighbor_distances: Corresponding distances from query point to K nearest neighbors. 
        """
        
        # If only one neighbor, return label of that neighbor 
        if len(neighbor_indices) == 1:
            return self.y_train[neighbor_indices[0]]
        
        # If no distances provided, set to ones 
        if neighbor_distances is None:
            neighbor_distances = np.ones(len(neighbor_indices))
        
        # YOUR CODE HERE
        raise NotImplementedError()
            
    def classify(self, x):
        """
        Given a query point, return the predicted label 
        
        :param x: a query point stored as an ndarray  
        """
        # YOUR CODE HERE
        raise NotImplementedError()
        
    def predict(self, X):
        """
        Given an ndarray of query points, return yhat, an ndarray of predictions 

        :param X: an (m x p) dimension ndarray of points to predict labels for 
        """
        # YOUR CODE HERE
        raise NotImplementedError()

**Part B [10 points]**: Modify the class above to implement an Unweighted KNN classifier.  There are three methods that you need to complete: 

- `predict`: Given an $m \times p$ matrix of validation data with $m$ examples each with $p$ features, return a length-$m$ vector of predicted labels by calling the `classify` function on each example. 
- `classify`: Given a single query example with $p$ features, return its predicted class label as a string using KNN by calling the `majority` function. 
- `majority`: Given an array of indices into the training set corresponding to the $K$ training examples that are nearest to the query point, return the majority label as a string.  If there is a tie for the majority label using $K$ nearest neighbors, reduce $K$ by 1 and try again.  Continue reducing $K$ until there is a winning label. 

**Notes**: 
- Don't even think about implementing nearest-neighbor search or any distance metrics yourself.  Instead, go read the documentation for Scikit-Learn's [BallTree](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.BallTree.html) object.  You will find that its implemented [query](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.BallTree.html#sklearn.neighbors.BallTree.query) method can do most of the heavy lifting for you. 
- **Do not** use Scikit-Learn's KNeighborsClassifier in this problem.  We're implementing this ourselves. 
- You don't need to worry about the `distance_weighted` flag until **Part C**, but we recommend reading ahead a bit. It might be good to think about your implementation of **Part C** before implementing **Part B**. 
- When you think you're done, execute the following cell to run 4 unit tests based on the example starting on Slide 15 of the KNN Lecture slides on Canvas.

In [ ]:
from tests import tests
tests.run_test_suite('prob 2A', KNN)

**Part C [5 points]**: Modify the `KNN` class to perform the distance-weighted KNN classification.
The so-called Distance-Weighted KNN classifier assigns weights to the nearest-neighbor training examples proportional to the inverse-distance from the training example to the query point.  Classification is performed by summing the weights associated with each class and predicting the class with the highest weighted-majority vote.  Mathematically we might describe the weighted-vote for a class $c$ as 

$$
\textrm{Weighted-Vote}(c) = \displaystyle\sum_{i \in {\cal N}_K} I(y_i = c) \times \dfrac{1}{\|{\bf x}_i - {\bf x}\|}
$$

A word of caution: it's certainly possible that a query point could be distance $0$ away from some training example.  If this happens your implementation should handle it gracefully and return the appropriate class label.   

When you think you're done, execute the following cell to run three final unit tests corresponding to the example on Slide 21 of the KNN Lecture slides on Canvas. Make sure that the changes you make in **Part C** do not affect the unit tests from **Part B**.   


In [ ]:
from tests import tests
tests.run_test_suite('prob 2B', KNN)

**Part D [8 points]**: Use your `KNN` class to perform weighted KNN on the validation data with $K=100$ and do the following: 

- **[4 points]** Create a **confusion matrix** (feel free to use the Scikit-Learn [confusion_matrix](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) function).  


In [ ]:
knn = KNN(X_train, y_train, K=100, distance_weighted=True)
yhat_valid = knn.predict(X_valid)
# YOUR CODE HERE
raise NotImplementedError()

- **[4 points]** Based on your confusion matrix, which crime types are most frequently misclassified as other crime types? 

YOUR ANSWER HERE

**Part E [12 points]**: **[4 points]** Create a plot of the accuracy of both Unweighted and Distance-Weighted KNN on the validation set on the same set of axes for various values of $K$. Your plot should show the approximate value of $K$ that maximizes the accuracy. Note that accuracy will be below 50%, but above 25% (i.e., better than random classification).

In [ ]:
acc = []
wacc = []
allks = [50, 100, 150, 200, 250, 300]

# YOUR CODE HERE
raise NotImplementedError()
    
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(12, 7))
ax.plot(allks, acc, marker="o", color="steelblue", lw=3, label="unweighted")
ax.plot(allks, wacc, marker="o", color="green", lw=3, label="weighted")
ax.set_xlabel("number neighbors", fontsize=16)
ax.set_ylabel("accuracy", fontsize=16)
ax.legend(loc="upper right")
plt.xticks(range(50, 301, 50))
ax.grid(alpha=0.25)

**[4 points]** Based on the plot, answer the following questions: 

- For general $K$, does Unweighted or Weighted KNN appear to perform better? 
- Which value of $K$ attains the best accuracy on the validation set? 

Open questions: Why do you think this is the case? How can you explain this?

YOUR ANSWER HERE

**[4 points]** If you had unrestricted access to data, how might you improve the model to try to increase classification accuracy?

YOUR ANSWER HERE

YOUR ANSWER HERE

### Optional survey.
***

We are always interested in your feedback. At the end of each homework, there is a simple anonymous feedback [survey](https://docs.google.com/forms/d/e/1FAIpQLScOhZRze8OcoWVgOf-FP5gJFiecJPVFkGnQSfC2zGrvUmHdrQ/viewform?usp=sf_link) to solicit your feedback for how to improve the course.